# [Module 2] Forecast 학습 데이터 준비 (Import Dataset)
- 이 노트북에서는 Module1에서 생성한 train_target_time_series.csv 파일을 가지고 Forecast가 학습을 할 수 있게 하는 작업을 합니다.  

아래 (1) 단계의 데이타 파일의 준비는 이전 노트북에서 준비가 되었고, 여기서는 (2) ~ (7) 의 과정을 수행 합니다.

![Fig.2.1.Forecast-Lifecycle](img/Fig.2.1.Forecast-Lifecycle.png)
**Source: By Kyoungtae Hwang**

상세하게 이 노트북은 아래와 같은 작업을 수행 합니다.<br>

- Create IAM role
    - forecast 서비스가 다른 서비스(예: S3)에 접근시 사용할 역할을 생성하고 권한을 부여 합니다.
    
    
- (2) 데이터 파일 S3에 업로드 (Upload the Target Data to S3)
    - 이전 노트북에서 만든 train_target_time_series.csv 파일을 S3에 업로드 합니다.


- (3) Create a dataset group
    - 전체 데이터 셋을 (Target Data Set, Related Data Set, Item Meta Data Set)을 담을 상위의 Dataset Group을 생성 합니다. 여기서는 Target Data Set 만을 담습니다.


- (4) Create schema 
    - 여기서는 Target Data Set의 컬럼 정보, 컬럼 타입을 정의하는 스키마 파일을 정의해서 Forecast서비스가 어떠한 데이타가 입력 되는지를 알게 합니다.


- (5) Create Target Time Series Dataset
    - 실제로 Target Data Set을 생성 합니다.


- (6) Update dataset group
    - 위에서 생성된 Target Data Set을 Dataset Group에 연결 시키는 작업을 합니다.


- (7) Create dataset import job
    - S3에 업로드 된 train_target_time_series.csv 파일을 Target Data Set에 Import하여 Forecast 서비스가 사용할 수 있게 합니다.
    
---    
이 과정은 약 8분 정도 소요 됩니다 **About 8 mins may be elapsed**


In [1]:
import boto3
from time import sleep
import os
import pandas as pd
import json
import time
import numpy as np

In [2]:
%store -r

## Project Name 및 Parameters

- Target Dataset 및 Target Dataset Group 의 이름을 지정 합니다.
- DATASET_FREQUENCY 를 Day로 설정 합니다. 참고로 Week로 한다면 "W"로 지정 합니다. 또한 TIMESTAMP_FORMAT 를 yyyy-mm-dd 형식으로 지정 합니다.


In [3]:
# Enter a project name
project = 'StoreItemDemand'
target_suffix = '_target'

suffix = str(np.random.uniform())[4:9]

target_datasetName= project+'Dataset' + suffix
target_datasetGroupName= project +'DatasetGroup'+ suffix

DATASET_FREQUENCY = "D" 
TIMESTAMP_FORMAT = "yyyy-MM-dd"

## 역할 생성 (Create role)

**이 작업을 하기 전에  이 노트북을 실행하는 SageMaker notebook instance 가 AmazonSageMakerFullAccess, AmazonS3FullAccess, AmazonForecastFullAccess, IAMFullAccess 4개의 정책을 가지고 있어야 합니다.**
만일 아래 셀에서 에러가 발생했다면, [AddPolicy](../0.0.Prerequisite/Prerequisite.md) 의 스크롤을 내려서 "3. Add IAM Policy (Permission)" 부분 부터 시작하여 권한 추가 해주세요. (Make sure that a role for SageMaker notebook instance has these policies attached such as AmazonSageMakerFullAccess, AmazonS3FullAccess, AmazonForecastFullAccess, IAMFullAccess)

이 부분은 ForecastRolePOC_XXX 역할을 생성하고, ForecastRolePOC_XXX 에게 AmazonForecastFullAccess, AmazonS3FullAccess 이 두개의 Policy(권한)을 부여 합니다. ForecastRolePOC_XXX 는 Forecast 서비스가 다른 서비스(예: S3) 에 접근시 사용합니다.



In [4]:
iam = boto3.client("iam")

# Put the role name
role_name = "ForecastRolePOC" + suffix
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "forecast.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like tåo use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role

# AmazonForecastFullAccess 추가
policy_arn = "arn:aws:iam::aws:policy/AmazonForecastFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)


# AmazonS3FullAccess 추가
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::057716757052:role/ForecastRolePOC13625


In [7]:
region = boto3.Session().region_name
session = boto3.Session(region_name=region)
forecast = session.client(service_name='forecast')

## (2) 데이터 파일 S3에 업로드 - 가장 상단 이미지의 (2) 과정 임

이전에 노트북에서 생성한 train_target_time_series.csv 파일을 S3에 업로드 합니다.

In [8]:
import sagemaker

s3_resource = boto3.resource('s3')
s3 = boto3.client('s3')

# if you want, replace with a name of your S3 bucket
# 버킷을 원하시면 원하시는 버킷 이름을 적으세요. 여기서는 디폴트 버킷을 사용합니다.
bucket_name = sagemaker.Session().default_bucket()  
# bucket_name = "<bucket name>" 

if s3_resource.Bucket(bucket_name).creation_date is None:
    # bucket is not existing 
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})    
else: 
    # Bucket exists
    print("bucket name is ", bucket_name)
    


bucket name is  sagemaker-ap-northeast-2-057716757052


In [9]:
# Upload Target File under a bucket folder
bucket_folder = project
s3_file_path = bucket_folder + "/" + train_time_series_filename
train_time_series_path = os.path.join('data', train_time_series_filename)

boto3.Session().resource('s3').Bucket(bucket_name).Object(s3_file_path).upload_file(train_time_series_path)
target_s3DataPath = "s3://"+bucket_name + "/" + s3_file_path    

print("Uploaded S3 File Path:\n", target_s3DataPath)

Uploaded S3 File Path:
 s3://sagemaker-ap-northeast-2-057716757052/StoreItemDemand/train_target_time_series.csv


In [10]:
! aws s3 ls {target_s3DataPath} --recursive

2020-08-22 13:21:09   10124591 StoreItemDemand/train_target_time_series.csv


## (3) Create Dataset Group

Dataset Group을 생성 합니다. 도메인은 CUSTOM을 사용 합니다.

In [11]:
create_dataset_group_response = forecast.create_dataset_group(
      DatasetGroupName= target_datasetGroupName,
      Domain="CUSTOM",
     )
target_datasetGroupArn = create_dataset_group_response['DatasetGroupArn']

dataset_group 의 생성 상태를 확인 합니다.

In [12]:
forecast.describe_dataset_group(DatasetGroupArn=target_datasetGroupArn)

{'DatasetGroupName': 'StoreItemDemandDatasetGroup13625',
 'DatasetGroupArn': 'arn:aws:forecast:ap-northeast-2:057716757052:dataset-group/StoreItemDemandDatasetGroup13625',
 'DatasetArns': [],
 'Domain': 'CUSTOM',
 'Status': 'ACTIVE',
 'CreationTime': datetime.datetime(2020, 8, 22, 13, 21, 10, 726000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 8, 22, 13, 21, 10, 726000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '0768e7c2-9e38-415c-b2a6-048560e81096',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 22 Aug 2020 13:21:12 GMT',
   'x-amzn-requestid': '0768e7c2-9e38-415c-b2a6-048560e81096',
   'content-length': '292',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## (4) Create schema

dataset schema를 생성 합니다.

In [13]:
# Specify the schema of your dataset here. Make sure the order of columns matches the raw data files.
schema ={
   "Attributes":[
      {
         "AttributeName":"timestamp",
         "AttributeType":"timestamp"
      },
      {
         "AttributeName":"item_id",
         "AttributeType":"string"
      },
      {
         "AttributeName":"store",
         "AttributeType":"string"
      },       
      {
         "AttributeName":"target_value",
         "AttributeType":"float"
      },
   ]
}

## (5) Create Target Time Sereis Dataset

In [14]:
response=forecast.create_dataset(
                    Domain="CUSTOM",
                    DatasetType='TARGET_TIME_SERIES',
                    DatasetName=target_datasetName,
                    DataFrequency=DATASET_FREQUENCY, 
                    Schema = schema
)

In [15]:
target_datasetArn = response['DatasetArn']
forecast.describe_dataset(DatasetArn=target_datasetArn)

{'DatasetArn': 'arn:aws:forecast:ap-northeast-2:057716757052:dataset/StoreItemDemandDataset13625',
 'DatasetName': 'StoreItemDemandDataset13625',
 'Domain': 'CUSTOM',
 'DatasetType': 'TARGET_TIME_SERIES',
 'DataFrequency': 'D',
 'Schema': {'Attributes': [{'AttributeName': 'timestamp',
    'AttributeType': 'timestamp'},
   {'AttributeName': 'item_id', 'AttributeType': 'string'},
   {'AttributeName': 'store', 'AttributeType': 'string'},
   {'AttributeName': 'target_value', 'AttributeType': 'float'}]},
 'EncryptionConfig': {},
 'Status': 'ACTIVE',
 'CreationTime': datetime.datetime(2020, 8, 22, 13, 21, 14, 976000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 8, 22, 13, 21, 14, 976000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '5da32938-bbd0-47cd-a467-dee37d4d7f8b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 22 Aug 2020 13:21:15 GMT',
   'x-amzn-requestid': '5da32938-bbd0-47cd-a467-dee37d4d7f8

## (6) Update dataset group

In [16]:
# Attach the Dataset to the Dataset Group:
forecast.update_dataset_group(
    DatasetGroupArn=target_datasetGroupArn, 
    DatasetArns=[target_datasetArn])

{'ResponseMetadata': {'RequestId': '5a76a8d5-e109-4c48-b0fb-50a04c12e74e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 22 Aug 2020 13:21:16 GMT',
   'x-amzn-requestid': '5a76a8d5-e109-4c48-b0fb-50a04c12e74e',
   'content-length': '2',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## (7) Create dataset import job 
S3에서 Target Data Set으로 데이터를 Import(다운로드) 합니다.

In [17]:
# Finally we can call import the dataset
datasetImportJobName = 'DSIMPORT_JOB_TARGET_WALMART' + suffix
ds_import_job_response=forecast.create_dataset_import_job(DatasetImportJobName=datasetImportJobName,
                                                          DatasetArn=target_datasetArn,
                                                          DataSource= {
                                                              "S3Config" : {
                                                                 "Path":target_s3DataPath,
                                                                 "RoleArn": role_arn
                                                              } 
                                                          },
                                                          TimestampFormat=TIMESTAMP_FORMAT
                                                         )

In [18]:
ds_target_import_job_arn=ds_import_job_response['DatasetImportJobArn']
print(ds_target_import_job_arn)

arn:aws:forecast:ap-northeast-2:057716757052:dataset-import-job/StoreItemDemandDataset13625/DSIMPORT_JOB_TARGET_WALMART13625


In [19]:
%%time

while True:
    dataImportStatus = forecast.describe_dataset_import_job(DatasetImportJobArn=ds_target_import_job_arn)['Status']
    print(dataImportStatus)
    if dataImportStatus != 'ACTIVE' and dataImportStatus != 'CREATE_FAILED':
        sleep(30)
    else:
        break

CREATE_PENDING
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
ACTIVE
CPU times: user 64.3 ms, sys: 5.74 ms, total: 70 ms
Wall time: 8min 31s


In [20]:
%store project
%store suffix
%store target_suffix
%store region
%store ds_target_import_job_arn
%store target_datasetGroupArn
%store target_datasetArn
%store bucket_name
%store bucket_folder
%store role_arn
%store role_name
%store validation_stores_sales

Stored 'project' (str)
Stored 'suffix' (str)
Stored 'target_suffix' (str)
Stored 'region' (str)
Stored 'ds_target_import_job_arn' (str)
Stored 'target_datasetGroupArn' (str)
Stored 'target_datasetArn' (str)
Stored 'bucket_name' (str)
Stored 'bucket_folder' (str)
Stored 'role_arn' (str)
Stored 'role_name' (str)
Stored 'validation_stores_sales' (DataFrame)
